##  파이프라인(Pipeline)

Scikit-learn의 `Pipeline`은 데이터 전처리(preprocessing) 단계와 모델 학습(modeling) 단계를 일련의 순차적인 과정으로 묶어주는 강력한 도구.

파이프라인을 사용하면 다음과 같은 이점이 있다.

  * **코드 간소화:** 여러 단계를 하나의 객체로 관리할 수 있다.
  * **오류 방지:** 학습 데이터와 테스트 데이터에 동일한 변환(transform)이 적용되도록 보장.
  * **재현성 향상:** 전체 과정을 명확하게 정의하여 실험의 재현성을 높인다.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

np.random.seed(42)
X = np.random.rand(100, 3) * 10  # 100개의 샘플, 3개의 특성
y = 3 * X[:, 0] - 2 * X[:, 1] + X[:, 2] + np.random.randn(100) * 0.5


# 훈련/테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 변경된 부분: make_pipeline 사용 ---
# 튜플 (이름, 변환기/추정기) 형태 대신 객체만 나열.
pipeline = make_pipeline(
    StandardScaler(),    # 첫 번째 단계: 특성 표준화 (변환기)
    LinearRegression()   # 두 번째 단계: 선형 회귀 모델 (추정기)
)
# ------------------------------------

print("--- 정의된 파이프라인 구조 (make_pipeline 사용) ---")
print(pipeline)
print("-------------------------------------------------")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# make_pipeline은 클래스 이름을 소문자로 자동 지정.
# StandardScaler -> 'standardscaler'
# LinearRegression -> 'linearregression'
regressor_model = pipeline.named_steps['linearregression'] 

print(f"학습된 회귀 계수(Coefficients): {regressor_model.coef_}")
print(f"평균 제곱 오차 (MSE): {mean_squared_error(y_test, y_pred):.4f}")

# 파이프라인에서 최종 추정기(LinearRegression 모델)를 인덱스 [1]로 얻어옵니다.
regressor_model_by_index = pipeline[1]

print(f"--- 인덱스로 얻어온 추정기 정보 ---")
print(f"객체 타입: {type(regressor_model_by_index)}")
print(f"학습된 계수 (Coefficients): {regressor_model_by_index.coef_}")

In [ ]:

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

np.random.seed(42)
X = np.random.rand(100, 3) * 10  # 100개의 샘플, 3개의 특성
y = 3 * X[:, 0] - 2 * X[:, 1] + X[:, 2] + np.random.randn(100) * 0.5
# 훈련/테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
steps = [
    ('scaler', StandardScaler()),       # 첫 번째 단계: 특성 표준화 (변환기)
    ('regressor', LinearRegression())  # 두 번째 단계: 선형 회귀 모델 (추정기)
]
pipeline = Pipeline(steps)

print("--- 정의된 파이프라인 구조 ---")
print(pipeline)
print("----------------------------")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

regressor_model = pipeline.named_steps['regressor']
print(f"학습된 회귀 계수(Coefficients): {regressor_model.coef_}")

### label , one hot encoding

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder
import numpy as np

# 훈련 데이터 생성
data = {
    '수치형_특징': [10, 20, 30, 40, 50],
    '범주형_특징': ['A', 'B', 'A', 'C', 'B'],
    '그대로_유지': [1, 0, 1, 0, 1]
}
X = pd.DataFrame(data)

print("--- 원본 데이터 (X) ---")
print(X)
print("-----------------------")

# 1. 적용할 전처리 (변환기)와 대상 컬럼 정의

# 수치형 변환 정의 (StandardScaler)
numeric_features = ['수치형_특징']
numeric_transformer = StandardScaler()

# 범주형 변환 정의 (OneHotEncoder)
categorical_features = ['범주형_특징']
# sparse_output=False 로 설정하면 numpy 배열 형태로 변환 결과를 얻기 쉬움
# categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
categorical_transformer = OrdinalEncoder()


# 2. ColumnTransformer 생성

preprocessor = ColumnTransformer(
    transformers=[
        # (이름, 변환기, 대상 컬럼 리스트)
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    # 나머지(remainder) 컬럼 처리 방법 정의
    remainder='passthrough' # '그대로_유지' 컬럼은 변환 없이 그대로 통과
)

print("\n--- 정의된 ColumnTransformer 구조 ---")
print(preprocessor)
print("-----------------------------------")

# ColumnTransformer를 데이터에 적용
X_transformed = preprocessor.fit_transform(X)

print("\n--- 변환된 데이터 (X_transformed) ---")
print(X_transformed)
print("--------------------------------------")

In [ ]:
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 가상 데이터 생성
data = {
    '성별': ['남', '여', '남', '여', '여', '남', '남', '여', '남', '여'],
    '나이': [30, 25, 40, 22, 35, 50, 28, 33, 45, 29],
    'target': [1, 0, 1, 0, 0, 1, 1, 0, 1, 0] # 예측할 숫자 (Label)
}
df = pd.DataFrame(data)

# 특성 (X)과 타겟 (y) 분리
# 라벨 인코딩할 특성만 선택 (여기서는 '성별'만 인코딩 대상)
X = df[['성별', '나이']]
y = df['target']

# 훈련/테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("--- 훈련 데이터 (X_train) 샘플 ---")
print(X_train.head())
print("--------------------------------")


# 1. 라벨 인코딩을 적용할 특성 리스트 정의
categorical_features = ['성별']
# 인코딩이 필요 없는 숫자형 특성 (Optional: OrdinalEncoder는 문자열에만 적용)
# numerical_features = ['나이']

# 2. 전처리 단계 정의 (ColumnTransformer 사용)
# '성별' 컬럼에만 OrdinalEncoder를 적용합니다.

preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), categorical_features)
    ],
    remainder='passthrough' # '성별'이 아닌 나머지 특성('나이')은 그대로 통과
)

# 3. 전체 파이프라인 정의
full_pipeline = Pipeline( steps=[
    ('preprocessor', preprocessor),                       # 첫 번째 단계: 데이터 전처리 (인코딩)
    ('classifier', DecisionTreeClassifier(random_state=42)) # 두 번째 단계: 의사 결정 나무 모델
])

print("\n--- 정의된 파이프라인 구조 ---")
print(full_pipeline)
print("----------------------------")

--- 훈련 데이터 (X_train) 샘플 ---
  성별  나이
0  남  30
7  여  33
2  남  40
9  여  29
4  여  35
--------------------------------

--- 정의된 파이프라인 구조 ---
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal_encoder',
                                                  OrdinalEncoder(), ['성별'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])
----------------------------


In [3]:
# 1. 훈련 데이터로 전체 파이프라인 학습
# preprocessor가 X_train의 '성별'을 인코딩하고,
# classifier가 인코딩된 데이터를 받아 학습합니다.
print("\n--- 파이프라인 학습 시작 ---")
full_pipeline.fit(X_train, y_train)
print("--- 파이프라인 학습 완료 ---")

# 2. 테스트 데이터로 예측
# X_test에도 동일한 인코딩을 적용 후 예측합니다.
y_pred = full_pipeline.predict(X_test)

# 3. 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"\n테스트 세트 정확도 (Accuracy): {accuracy:.2f}")

# 4. 인코딩 결과 확인 (선택 사항)
# 파이프라인을 거친 후 '성별' 특성의 값이 어떻게 변환되었는지 확인
X_test_transformed = full_pipeline.named_steps['preprocessor'].transform(X_test)

# (주의: ColumnTransformer는 배열(Numpy array)을 반환하며, 컬럼 순서가 바뀔 수 있습니다.)
# 여기서는 인코딩된 '성별'이 첫 번째 컬럼에 위치합니다.
print("\n--- 테스트 데이터 변환 후 (성별 인코딩 결과) ---")
# '성별' 특성만 인코딩된 값 출력:
print(X_test_transformed[:, 0]) 
# (OrdinalEncoder는 기본적으로 학습 데이터에 나타난 순서대로 0, 1, 2... 숫자를 부여합니다.)


--- 파이프라인 학습 시작 ---
--- 파이프라인 학습 완료 ---

테스트 세트 정확도 (Accuracy): 1.00

--- 테스트 데이터 변환 후 (성별 인코딩 결과) ---
[0. 1. 0.]


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 데이터 준비 (이전 예제와 동일)
data = {
    '성별': ['남', '여', '남', '여', '여', '남', '남', '여', '남', '여'],
    '나이': [30, 25, 40, 22, 35, 50, 28, 33, 45, 29],
    'target': [1, 0, 1, 0, 0, 1, 1, 0, 1, 0] 
}
df = pd.DataFrame(data)

X = df[['성별', '나이']]
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. One-Hot 인코딩을 적용할 특성 리스트 정의
categorical_features = ['성별']

# 2. 전처리 단계 정의 (ColumnTransformer 사용)
# '성별' 컬럼에만 OneHotEncoder를 적용합니다.
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # '성별'이 아닌 나머지 특성('나이')은 그대로 통과
)

# 3. 전체 파이프라인 정의
full_pipeline_ohe = Pipeline(steps=[
    ('preprocessor', preprocessor),                       # 첫 번째 단계: 데이터 전처리 (One-Hot 인코딩)
    ('classifier', DecisionTreeClassifier(random_state=42)) # 두 번째 단계: 의사 결정 나무 모델
])

print("--- 정의된 One-Hot 파이프라인 구조 ---")
print(full_pipeline_ohe)
print("-----------------------------------")

# 1. 훈련 데이터로 전체 파이프라인 학습
print("\n--- 파이프라인 학습 시작 ---")
full_pipeline_ohe.fit(X_train, y_train)
print("--- 파이프라인 학습 완료 ---")

# 2. 테스트 데이터로 예측
y_pred = full_pipeline_ohe.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n테스트 세트 정확도 (Accuracy): {accuracy:.2f}")

# 3. One-Hot 인코딩 결과 확인
X_test_transformed = full_pipeline_ohe.named_steps['preprocessor'].transform(X_test)

# (OneHotEncoder의 결과는 기본적으로 Sparse Matrix일 수 있지만, 작은 데이터에서는 Dense Matrix로 보일 수 있습니다.)
# X_test_transformed는 이제 '성별_남', '성별_여', '나이' 순서로 3개의 컬럼을 가집니다.

print("\n--- 테스트 데이터 변환 후 (One-Hot 인코딩 결과) ---")
# 상위 2개의 행 출력
print(X_test_transformed[:2])

# 변환된 결과를 해석하면:
# 첫 번째 컬럼: 성별이 '남'이면 1, 아니면 0
# 두 번째 컬럼: 성별이 '여'이면 1, 아니면 0
# 세 번째 컬럼: '나이' 값 (passthrough 된 특성)

--- 정의된 One-Hot 파이프라인 구조 ---
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['성별'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])
-----------------------------------

--- 파이프라인 학습 시작 ---
--- 파이프라인 학습 완료 ---

테스트 세트 정확도 (Accuracy): 1.00

--- 테스트 데이터 변환 후 (One-Hot 인코딩 결과) ---
[[ 1.  0. 45.]
 [ 0.  1. 25.]]
